# Install libs

In [1]:
!pip install vibdata==1.1.1 signalAI==0.0.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00:00:010:01


# Import Libs

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import numpy.typing as npt
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

import vibdata.raw as raw_datasets
from vibdata.deep.DeepDataset import DeepDataset, convertDataset
from vibdata.deep.signal.transforms import (
    Sequential,
    SplitSampleRate,
    FeatureExtractor,
    FilterByValue,
    Split
)
from vibdata.deep.signal.core import SignalSample

# from signalAI.experiments.torch_data import DeepLearningExperiment
from signalAI.experiments.features_1d import Features1DExperiment
from signalAI.utils.group_dataset import GroupDataset
from signalAI.utils.fold_idx_generator import (
    FoldIdxGeneratorUnbiased,
    FoldIdxGeneratorBiased,
)

class GroupCWRULoad(GroupDataset):
    @staticmethod
    def _assigne_group(sample: SignalSample) -> int:
        return sample["metainfo"]["load"]

class GroupCWRUSeverity(GroupDataset):
    @staticmethod
    def _assigne_group(sample: SignalSample) -> int:
        severity = sample["metainfo"]["fault_size"]

        match severity:
            case 0.0:
                return sample["metainfo"]["load"]
            case 0.007:
                return 0
            case 0.014:
                return 1
            case 0.021:
                return 2
            case 0.028:
                return 3

        return None

# Deep Learning Experiments

## Import CRWU dataset

In [3]:
raw_root_dir = "../data/raw_data/cwru"
raw_dataset = raw_datasets.CWRU_raw(raw_root_dir, download=True)

Cached downloading...
Hash: md5:d7d3042161080fc82e99d78464fa2914
From (original): https://drive.google.com/uc?id=1G2vfms1QDlkdzqL_LAQdMIQAoludxBNj
From (redirected): https://drive.google.com/uc?id=1G2vfms1QDlkdzqL_LAQdMIQAoludxBNj&confirm=t&uuid=341da9c5-55d9-4b58-90a0-ee7dfb779350
To: ../data/raw_data/cwru/CWRU_raw/CWRU.zip
100%|██████████| 245M/245M [00:02<00:00, 82.8MB/s] 


## Time domain

### 12k SampleRate

In [4]:
transforms_time = Sequential(
    [
        SplitSampleRate(),
        Split(1024)
    ]
)
print(transforms_time)

Sequential(transforms=[SplitSampleRate(), Split(window_size=1024)])


In [5]:
deep_root_dir_time = "../data/deep_data/deep_learning"
deep_dataset_time = convertDataset(raw_dataset,filter=FilterByValue(on_field="sample_rate", values=12000),transforms=transforms_time, dir_path=deep_root_dir_time, batch_size=32)

Transformando


Converting CWRU: 100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


## Generate Unbiased Folds

In [6]:
folds_singleround_deep = FoldIdxGeneratorUnbiased(deep_dataset_time, GroupCWRULoad , dataset_name="CWRU12k_deep").generate_folds()
folds_singleround_deep

Grouping dataset: 100%|██████████| 27720/27720 [00:04<00:00, 6675.12sample/s]


array([0., 0., 0., ..., 3., 3., 3.])

## Generate Biased Folds

In [7]:
folds_singleround_deep_biased = FoldIdxGeneratorBiased(deep_dataset_time, dataset_name="CWRU12k_deep", n_folds=4).generate_folds()
folds_singleround_deep_biased

array([0, 2, 2, ..., 1, 0, 2])

## DeepLearning Experiments

### Utils

In [8]:
# vibclassifier/experiments/base.py
from abc import ABC, abstractmethod
import json
from typing import Optional, Dict, Any
from vibdata.raw.base import RawVibrationDataset
from vibdata.deep.signal.transforms import Transform

class Experiment(ABC):
    """Classe base abstrata para todos os experimentos de classificação de vibração."""

    def __init__(
        self,
        name: str,
        description: str,
        dataset: Optional[RawVibrationDataset] = None,
        data_transform: Optional[Transform] = None,
        feature_selector = None,
        model = None
    ):
        """
        Inicializa o experimento.

        Args:
            name: Nome identificador do experimento
            description: Descrição detalhada do experimento
            dataset: Conjunto de dados de vibração
            data_transform: Transformação a ser aplicada nos dados brutos
            data_division_method: Método de divisão dos dados (e.g., 'kfold', 'holdout')
            data_division_params: Parâmetros para o método de divisão
            feature_selector: Seletor de features (para experimentos com extração)
            model: Modelo de machine learning/deep learning
        """
        self.name = name
        self.description = description
        self.dataset = dataset
        self.data_transform = data_transform
        self.feature_selector = feature_selector
        self.model = model

        # Resultados serão armazenados aqui
        self.results = {}

    @abstractmethod
    def prepare_data(self):
        """Prepara os dados para o experimento."""
        pass

    @abstractmethod
    def run(self):
        """Executa o experimento completo."""
        pass

    def save_results(self, filepath: str):
        """Salva os resultados do experimento."""
        # Implementação básica - pode ser extendida
        with open(filepath, 'w') as f:
            json.dump(self.results, f)

    def load_results(self, filepath: str):
        """Carrega resultados de um experimento anterior."""
        with open(filepath, 'r') as f:
            self.results = json.load(f)

    def __str__(self):
        return f"Experiment: {self.name}\nDescription: {self.description}"

In [9]:
# vibclassifier/experiments/deep_torch.py
import os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import numpy as np
from typing import List, Dict, Optional, Tuple, Union
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from signalAI.utils.metrics import calculate_metrics
from signalAI.utils.experiment_result import ExperimentResults, FoldResults
import copy

class TorchVibrationDataset(Dataset):
    """Wrapper to convert dataset samples into Torch tensors."""
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Função auxiliar KL
def kl_divergence(rho, rho_hat):
    rho_hat = torch.mean(rho_hat, dim=0)
    rho = torch.tensor([rho] * len(rho_hat), device=rho_hat.device)
    epsilon = 1e-7
    term1 = rho * torch.log((rho + epsilon) / (rho_hat + epsilon))
    term2 = (1 - rho) * torch.log((1 - rho + epsilon) / (1 - rho_hat + epsilon))
    return torch.sum(term1 + term2)

class DeepLearningExperiment(Experiment):
    def __init__(
        self,
        name: str,
        description: str,
        dataset,
        data_fold_idxs: List[int],
        model: nn.Module,
        criterion: Optional[nn.Module] = None,
        # Parâmetros adaptados para o autoencoder
        reconstruction_criterion: Optional[nn.Module] = None,
        recon_loss_weight: float = 1.0,
        sparsity_target: Optional[float] = None,
        sparsity_weight: float = 0.0,
        pretrain_epochs: int = 0, # Épocas de treinamento do autoencoder
        optimizer_class: Optional[torch.optim.Optimizer] = optim.Adam,
        batch_size: int = 32,
        lr: float = 1e-3,
        num_epochs: int = 20, # Épocas de treino do classificador
        val_split: float = 0.2,
        output_dir: str = "results_torch",
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        **kwargs
    ):
        super().__init__(name, description, dataset, model=model, **kwargs)
        self.data_fold_idxs = data_fold_idxs
        self.output_dir = Path(output_dir)
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.pretrain_epochs = pretrain_epochs
        self.val_split = val_split
        self.device = device
        self.optimizer_class = optimizer_class
        self.lr = lr
        self.criterion = criterion if criterion is not None else nn.CrossEntropyLoss()
        
        self.reconstruction_criterion = reconstruction_criterion
        self.recon_loss_weight = recon_loss_weight
        self.sparsity_target = sparsity_target
        self.sparsity_weight = sparsity_weight
        
        self.is_sae_task = self.sparsity_target is not None and self.sparsity_weight > 0.0
        # Define tipo do AutoEncoder utilizado
        self.is_autoencoder_task = reconstruction_criterion is not None or self.is_sae_task or "AE1D" in model.__class__.__name__

        if self.is_sae_task and self.reconstruction_criterion is None:
             print("Warning: SAE task detected but no reconstruction_criterion. Defaulting to MSELoss.")
             self.reconstruction_criterion = nn.MSELoss()

        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model) 
        
        # Guarda encoder e decoder
        self.original_model = model.module if isinstance(model, nn.DataParallel) else model

        self.n_outer_folds = len(np.unique(data_fold_idxs))
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.prepare_data()

    def prepare_data(self):
        features, labels = [], []
        for sample in self.dataset:
            features.append(sample['signal'][0]) 
            labels.append(sample['metainfo']['label'])
        self.X = np.array(features)
        self.y = np.array(labels)

    def _train_one_fold(
        self, X_train, y_train, X_test, y_test, fold_idx: int
    ) -> FoldResults:
        
        train_dataset = TorchVibrationDataset(X_train, y_train)
        test_dataset = TorchVibrationDataset(X_test, y_test)
        val_size = int(self.val_split * len(train_dataset))
        train_size = len(train_dataset) - val_size
        train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
        
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

        model = copy.deepcopy(self.model.to(self.device))
        model_core = model.module if isinstance(model, nn.DataParallel) else model

        # Verificação da estrutura de AE (encoder, decoder, classifier)
        has_ae_structure = hasattr(model_core, 'encoder') and hasattr(model_core, 'decoder') and hasattr(model_core, 'classifier')

        # ============================================================
        # TREINO DO AUTOENCODER
        # ============================================================
        if self.is_autoencoder_task and self.pretrain_epochs > 0 and has_ae_structure:
            print(f"[Fold {fold_idx}] AutoEncoder training ({self.pretrain_epochs} epochs)...")
            
            # Otimizador Encoder + Decoder
            optimizer_ae = self.optimizer_class([
                {'params': model_core.encoder.parameters()},
                {'params': model_core.decoder.parameters()}
            ], lr=self.lr)

            for epoch in range(self.pretrain_epochs):
                model.train()
                running_recon_loss = 0.0
                
                for xb, _ in train_loader:
                    xb = xb.to(self.device)
                    input_data = xb

                    if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                        xb = xb.unsqueeze(1)
                    elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                         side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                    optimizer_ae.zero_grad()
                    outputs = model(xb) # (class, recon, [sparsity])

                    if isinstance(outputs, tuple):
                        # Foco na reconstrução
                        reconstruction = outputs[1] 
                        
                        loss = self.reconstruction_criterion(reconstruction, input_data)
                        
                        # Adiciona esparsidade se for SAE
                        if self.is_sae_task and len(outputs) > 2:
                            latent_features = outputs[2]
                            loss += self.sparsity_weight * kl_divergence(self.sparsity_target, latent_features)
                        
                        loss.backward()
                        optimizer_ae.step()
                        running_recon_loss += loss.item() * input_data.size(0)
                
                avg_recon_loss = running_recon_loss / len(train_loader.dataset)
                if (epoch + 1) % 5 == 0 or epoch == 0:
                    print(f"  [Pre-train] Epoch {epoch+1}/{self.pretrain_epochs} Recon Loss: {avg_recon_loss:.4f}")

        # ============================================================
        # TREINO DO CLASSIFICADOR
        # ============================================================
        print(f"[Fold {fold_idx}] Classifier training ({self.num_epochs} epochs)...")

        # Define otimizador para a fase supervisionada
        if has_ae_structure and self.is_autoencoder_task:
            # Se for AE: Treina Encoder + Classifier (Decoder congelado ou ignorado pelo otimizador)
            optimizer_clf = self.optimizer_class([
                {'params': model_core.encoder.parameters()},
                {'params': model_core.classifier.parameters()}
            ], lr=self.lr)
        else:
            # Se for MLP/CNN padrão: Treina todos os parâmetros
            optimizer_clf = self.optimizer_class(model.parameters(), lr=self.lr)

        train_losses, val_losses = [], []
        
        for epoch in range(self.num_epochs):
            epoch_start = time.time()
            model.train()
            running_loss = 0.0
            
            for xb, yb in train_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                
                # Ajuste de shape
                if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                     xb = xb.unsqueeze(1)
                elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                     side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                optimizer_clf.zero_grad()
                outputs = model(xb)

                # Cálculo da perda apenas de CLASSIFICAÇÃO
                if isinstance(outputs, tuple):
                    classification_output = outputs[0] # Pega apenas a classificação
                else:
                    classification_output = outputs # Modelo padrão

                loss = self.criterion(classification_output, yb)
                
                loss.backward()
                optimizer_clf.step()
                running_loss += loss.item() * xb.size(0)

            avg_train_loss = running_loss / len(train_loader.dataset)

            # Validação
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(self.device), yb.to(self.device)
                    # Ajuste de shape
                    if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                         xb = xb.unsqueeze(1)
                    elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                         side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                    outputs = model(xb)
                    
                    if isinstance(outputs, tuple):
                        classification_output = outputs[0]
                    else:
                        classification_output = outputs

                    loss = self.criterion(classification_output, yb)
                    val_loss += loss.item() * xb.size(0)

            avg_val_loss = val_loss / len(val_loader.dataset)
            
            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)
            
            epoch_time = time.time() - epoch_start
            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(f"  [Supervised] Epoch {epoch+1}/{self.num_epochs} Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Time: {epoch_time:.2f}s")

        plt.figure()
        plt.plot(train_losses, label="Train Loss (Clf)")
        plt.plot(val_losses, label="Val Loss (Clf)")
        plt.legend(); plt.title(f"Loss Curve - Fold {fold_idx}")
        plt.savefig(os.path.join(self.dir_path, f"loss_curve_fold{fold_idx}_{self.start_time}.png")); plt.close()
        
        torch.save(model_core.state_dict(), os.path.join(self.dir_path, f"model_fold{fold_idx}.pt"))

        # Teste
        y_true, y_pred, y_proba = [], [], []
        model.eval()
        with torch.no_grad():
            for xb, yb in test_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                     xb = xb.unsqueeze(1)
                elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                     side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                outputs = model(xb)
                if isinstance(outputs, tuple):
                    classification_output = outputs[0]
                else:
                    classification_output = outputs

                probs = torch.softmax(classification_output, dim=1)
                preds = torch.argmax(probs, dim=1)
                y_true.extend(yb.cpu().numpy()); y_pred.extend(preds.cpu().numpy()); y_proba.extend(probs.cpu().numpy())

        metrics = calculate_metrics(np.array(y_true), np.array(y_pred), np.array(y_proba))
        return FoldResults(fold_idx, np.array(y_true), np.array(y_pred), np.array(y_proba), metrics)

    def run(self) -> ExperimentResults:
        self.start_time = time.strftime("%Y%m%d_%H%M%S")
        self.dir_path = os.path.join(self.output_dir, f"results_{self.name}_{self.start_time}")
        os.makedirs(self.dir_path, exist_ok=True)

        results = ExperimentResults(
            experiment_name=self.name, description=self.description,
            model_name=self.original_model.__class__.__name__, feature_names=None,
            config={'n_outer_folds': self.n_outer_folds, 'pretrain_epochs': self.pretrain_epochs, 
                    'finetune_epochs': self.num_epochs, 'batch_size': self.batch_size, 'lr': self.lr}
        )

        for outer_fold in range(self.n_outer_folds):
            print(f"\n=== Outer Fold {outer_fold+1}/{self.n_outer_folds} ===")
            train_mask = self.data_fold_idxs != outer_fold
            test_mask = self.data_fold_idxs == outer_fold
            
            try:
                fold_result = self._train_one_fold(self.X[train_mask], self.y[train_mask], self.X[test_mask], self.y[test_mask], outer_fold)
                results.add_fold_result(fold_result)
                print(f"  Result: Acc={fold_result.metrics['accuracy']:.4f}, F1={fold_result.metrics['f1']:.4f}")
            except Exception as e:
                print(f"Error in fold {outer_fold}: {e}")
                import traceback; traceback.print_exc()

        results.calculate_overall_metrics()
        results.save_json(os.path.join(self.dir_path, f"results.json"))
        print("\n=== Final Results ===")
        print(f"Mean Accuracy: {results.overall_metrics['accuracy']:.4f}")
        return results

### MLP

In [ ]:
class MLP1D(nn.Module):
    """
    Implementação do MLP 1D conforme benchmark (Zhao et al., 2020).
    Arquitetura: 1024 -> 512 -> 256 -> 128 -> 64 -> num_classes
    """
    def __init__(self, input_length: int = 1024, num_classes: int = 4):
        super().__init__()
        
        self.fc2 = nn.Sequential(
            nn.Linear(input_length, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True)
        )

        self.fc3 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True)
        )

        self.fc4 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True)
        )

        self.fc5 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True)
        )

        self.fc6 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True)
        )

        self.fc7 = nn.Sequential(
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # Achata o input para (Batch_Size, Features) permitindo entradas (N, 1, 1024) ou (N, 1024)
        out = torch.flatten(x, 1)
                
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        out = self.fc5(out)
        out = self.fc6(out)
        out = self.fc7(out)
        
        return out

In [13]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = MLP1D(input_length=input_length, num_classes=num_classes)

exp = DeepLearningExperiment(
    name="mlp1d_vibration",
    description="1D MLP for vibration signals",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,  # numpy array of fold indexes
    model=model,
    batch_size=64,
    lr=1e-3,
    pretrain_epochs=0,
    num_epochs=100
)

results = exp.run()

Input length: 1024, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] Classifier training (100 epochs)...
  [Supervised] Epoch 1/100 Train Loss: 0.9000, Val Loss: 0.7154, Time: 7.78s
  [Supervised] Epoch 5/100 Train Loss: 0.1883, Val Loss: 0.5380, Time: 7.02s
  [Supervised] Epoch 10/100 Train Loss: 0.1104, Val Loss: 0.6780, Time: 6.92s
  [Supervised] Epoch 15/100 Train Loss: 0.0800, Val Loss: 0.6976, Time: 7.24s
  [Supervised] Epoch 20/100 Train Loss: 0.0590, Val Loss: 0.6264, Time: 7.53s
  [Supervised] Epoch 25/100 Train Loss: 0.0499, Val Loss: 0.6673, Time: 7.79s
  [Supervised] Epoch 30/100 Train Loss: 0.0452, Val Loss: 0.8382, Time: 8.51s
  [Supervised] Epoch 35/100 Train Loss: 0.0436, Val Loss: 0.5928, Time: 7.30s
  [Supervised] Epoch 40/100 Train Loss: 0.0362, Val Loss: 0.6686, Time: 7.66s
  [Supervised] Epoch 45/100 Train Loss: 0.0393, Val Loss: 0.6035, Time: 7.71s
  [Supervised] Epoch 50/100 Train Loss: 0.0232, Val Loss: 0.7527, Time: 7.70s
  [Supervised] Epoch 55/100 Train Loss: 

### 1D AutoEncoder

In [14]:
class AE1D(nn.Module):
    """
    Implementação do Autoencoder 1D conforme benchmark (Zhao et al., 2020).
    Arquitetura: 1024 -> 512 -> 256 -> 128 -> 64 (Latent) -> 128 -> 256 -> 512 -> 1024.
    """
    def __init__(self, input_length: int = 1024, latent_dim: int = 64, num_classes: int = 4, dropout_rate: float = 0.4):
        super(AE1D, self).__init__()
        
        # --- Encoder ---
        # Reduz a dimensão progressivamente para extrair features
        # O uso de Dropout aqui cria um "Denoising effect" implícito nas features latentes
        self.encoder = nn.Sequential(
            # Camada 1: 1024 -> 512 (Redução imediata conforme padrão do artigo)
            nn.Linear(input_length, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            # Camada 2: 512 -> 256
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            # Camada 3: 256 -> 128
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            # Camada Latente: 128 -> 64
            nn.Linear(128, latent_dim)
        )

        # --- Decoder ---
        # Espelha o encoder para reconstrução. 
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),

            # Camada final de reconstrução
            nn.Linear(512, input_length) # Sem ativação final (Linear) devido dados padronizados (StandardScaler)
        )

        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        # Garante (Batch, Features)
        x = torch.flatten(x, 1)
        
        # Encoder
        latent_features = self.encoder(x)
        
        # Decoder (Reconstrução)
        reconstruction = self.decoder(latent_features)
        
        # Classifier (Diagnóstico)
        classification_output = self.classifier(latent_features)
        
        return classification_output, reconstruction, latent_features

In [15]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = AE1D(input_length=input_length, latent_dim=64, num_classes=num_classes)

# Defina os critérios
classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

exp = DeepLearningExperiment(
    name="ae1d_vibration_combined_loss",
    description="1D AE for vibration signals (Combined Loss)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # critério para treinamento do autoencoder
    criterion=classification_criterion,                # critério para treinamento do classificador
    pretrain_epochs=10,  # Treina Encoder+Decoder apenas com MSELoss
    num_epochs=10,       # Treina Encoder+Classifier apenas com CrossEntropy
    recon_loss_weight=1.0, # recon_loss_weight pode ser 1.0 (pois é a única loss na fase 1)
    batch_size=64,
    lr=1e-3
)

results = exp.run()

Input length: 1024, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.1308
  [Pre-train] Epoch 5/10 Recon Loss: 0.1184
  [Pre-train] Epoch 10/10 Recon Loss: 0.1163
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.1428, Val Loss: 0.9492, Time: 4.89s
  [Supervised] Epoch 5/10 Train Loss: 0.6120, Val Loss: 0.6089, Time: 4.46s
  [Supervised] Epoch 10/10 Train Loss: 0.3493, Val Loss: 0.5042, Time: 4.49s
  Result: Acc=0.5215, F1=0.5081

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.1421
  [Pre-train] Epoch 5/10 Recon Loss: 0.1268
  [Pre-train] Epoch 10/10 Recon Loss: 0.1239
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.1332, Val Loss: 0.9651, Time: 4.50s
  [Supervised] Epoch 5/10 Train Loss: 0.6621, Val Loss: 0.6743, Time: 4.45s
  [Supervised] Epoch 10/10 Train Loss: 0.4583, Val Loss: 0.6

### 1D Sparse AutoEncoder

In [16]:
class SAE1D(nn.Module):
    """
    Implementação do Sparse Autoencoder 1D (Zhao et al., 2020).
    Arquitetura: 1024 -> 512 -> 256 -> 128 -> 64 (Latent).
    """
    def __init__(self, input_length: int = 1024, latent_dim: int = 64, num_classes: int = 4):
        super(SAE1D, self).__init__()
        
        # --- Encoder ---
        self.encoder = nn.Sequential(
            nn.Linear(input_length, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),

            nn.Linear(128, latent_dim)
        )

        # Ativação específica para impor restrição (0,1) para a penalidade de esparsidade (KL Divergence)
        self.sparsity_activation = nn.Sigmoid()

        # --- Decoder ---
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),

            nn.Linear(512, input_length)
        )

        # Classificador para a fase de Fine-Tuning
        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        # Garante achatamento (Batch, 1024)
        x = torch.flatten(x, 1)

        # 1. Codificação
        features = self.encoder(x)
        
        # 2. Aplicação da não-linearidade para Esparsidade (Latent Space)
        latent_features = self.sparsity_activation(features)

        # 3. Decodificação (Reconstrução) baseada nas features esparsas
        reconstruction = self.decoder(latent_features)

        # 4. Classificação (Diagnóstico)
        classification_output = self.classifier(latent_features)

        return classification_output, reconstruction, latent_features

In [17]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")

model = SAE1D(input_length=input_length, latent_dim=64, num_classes=num_classes)

# loss criterions
classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

exp = DeepLearningExperiment(
    name="sae1d_vibration_sequential",
    description="1D Sparse AE for vibration signals (Sequential Training)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # ae train
    criterion=classification_criterion,                # classifier train
    pretrain_epochs=10,  # Treina Encoder+Decoder com MSE + Esparsidade KL
    num_epochs=10,       # Treina Encoder+Classifier com CrossEntropy
    sparsity_target=0.05,  # (Rho) Nível de esparsidade desejado (ex: 5% dos neurônios ativos) 
    sparsity_weight=1.0,   # (Beta) Peso da penalidade KL na perda total
    recon_loss_weight=1.0, # Peso da reconstrução (geralmente 1.0 na fase de pré-treino puro)
    batch_size=64,
    lr=1e-3
)

results = exp.run()

Input length: 1024, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 2.3288
  [Pre-train] Epoch 5/10 Recon Loss: 0.1229
  [Pre-train] Epoch 10/10 Recon Loss: 0.1156
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.0765, Val Loss: 0.8768, Time: 4.74s
  [Supervised] Epoch 5/10 Train Loss: 0.2076, Val Loss: 0.5060, Time: 3.92s
  [Supervised] Epoch 10/10 Train Loss: 0.0992, Val Loss: 0.5595, Time: 4.78s
  Result: Acc=0.5212, F1=0.5164

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 2.3976
  [Pre-train] Epoch 5/10 Recon Loss: 0.1331
  [Pre-train] Epoch 10/10 Recon Loss: 0.1253
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.0840, Val Loss: 0.8885, Time: 3.98s
  [Supervised] Epoch 5/10 Train Loss: 0.2736, Val Loss: 0.6478, Time: 3.93s
  [Supervised] Epoch 10/10 Train Loss: 0.1415, Val Loss: 0.6

### 1D Denoising AutoEncoder

In [18]:
class DAE1D(nn.Module):
    """
    Implementação do Denoising Autoencoder (DAE) 1D.
    Segue a arquitetura de referência do benchmark CWRU (Zhao et al., 2020).
    Arquitetura: 1024 -> 512 -> 256 -> 128 -> 64 (Latent).
    """
    def __init__(self, input_length: int = 1024, latent_dim: int = 64, num_classes: int = 4, noise_factor: float = 0.5):
        """
        Args:
            noise_factor: Desvio padrão do ruído Gaussiano adicionado (sigma).
        """
        super(DAE1D, self).__init__()
        self.noise_factor = noise_factor
        
        # --- Encoder ---
        # 1024 -> 512 -> 256 -> 128 -> 64
        self.encoder = nn.Sequential(
            nn.Linear(input_length, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, latent_dim)
        )

        # --- Decoder ---
        # 64 -> 128 -> 256 -> 512 -> 1024
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Linear(512, input_length)
        )

        # --- Classificador (Fine-tuning) ---
        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        # Garante (Batch, 1024)
        x = torch.flatten(x, 1)

        # --- Passo de Denoising (Apenas no Treino) ---
        if self.training:
            # Ruído Gaussiano (Normal): Média 0, Desvio Padrão = noise_factor
            # O artigo geralmente usa ruído aditivo.
            noise = torch.randn_like(x) * self.noise_factor
            x_noisy = x + noise
        else:
            x_noisy = x
        
        # 1. Encode (recebe entrada ruidosa)
        latent_features = self.encoder(x_noisy)
        
        # 2. Decode (tenta reconstruir)
        reconstruction = self.decoder(latent_features)
        
        # 3. Classify (diagnóstico)
        classification_output = self.classifier(latent_features)

        # Retorna x_noisy opcionalmente para visualização se necessário, 
        # mas para o treino precisamos: class, recon
        return classification_output, reconstruction

In [19]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")

# noise_factor controla o ruído
model = DAE1D(input_length=input_length, latent_dim=64, num_classes=num_classes, noise_factor=0.5)

classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

# 4. Configuração do Experimento
exp = DeepLearningExperiment(
    name="dae1d_vibration_sequential",
    description="1D Denoising AE for vibration signals (Sequential Training)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # ae train
    criterion=classification_criterion,                # classifier train
    pretrain_epochs=10,  # Treina Encoder+Decoder com MSELoss (DAE)
    num_epochs=10,       # Treina Encoder+Classifier com CrossEntropy
    recon_loss_weight=1.0,
    batch_size=64, # Batch grande ajuda na estabilidade do AE
    lr=1e-3
)

# 5. Execução
results = exp.run()

Input length: 1024, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.1272
  [Pre-train] Epoch 5/10 Recon Loss: 0.1128
  [Pre-train] Epoch 10/10 Recon Loss: 0.1070
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.2620, Val Loss: 1.2533, Time: 4.16s
  [Supervised] Epoch 5/10 Train Loss: 1.1416, Val Loss: 1.3213, Time: 4.92s
  [Supervised] Epoch 10/10 Train Loss: 1.0873, Val Loss: 1.3111, Time: 4.29s
  Result: Acc=0.2688, F1=0.2734

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.1401
  [Pre-train] Epoch 5/10 Recon Loss: 0.1237
  [Pre-train] Epoch 10/10 Recon Loss: 0.1184
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.2601, Val Loss: 1.2528, Time: 4.81s
  [Supervised] Epoch 5/10 Train Loss: 1.1231, Val Loss: 1.3713, Time: 4.00s
  [Supervised] Epoch 10/10 Train Loss: 1.0726, Val Loss: 1.3

### 1D CNN

In [21]:
class CNN1D(nn.Module):
    """
    Implementação da 1D CNN baseada no Benchmark (Zhao et al., 2020).
    Arquitetura: 4 Camadas de Convolução + 3 Camadas Densas.
    """
    def __init__(self, input_length: int = 1024, num_classes: int = 10):
        super(CNN1D, self).__init__()
        
        # Bloco de Extração de Features 1
        # Convoluções iniciais para capturar padrões brutos
        self.features1 = nn.Sequential(
            # Camada 1: Kernel médio (15) para capturar tendências locais
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=15, stride=1, padding=0),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            
            # Camada 2: Kernel fino (3) para refinar features
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),

            # Pooling agressivo para redução de dimensão
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # Bloco de Extração de Features 2        
        self.features2 = nn.Sequential(
            # Camada 3: Aumenta canais para 64
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            
            # Camada 4: Aumenta canais para 128
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            # Adaptive Pool garante que a saída seja sempre (Batch, 128, 4)
            # Isso independente de pequenas variações no input_length
            nn.AdaptiveMaxPool1d(output_size=4)
        )
        
        # Classificador (MLP Head)
        # Input: 128 canais * 4 pontos = 512 features
        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            # FC 1
            nn.Linear(128 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5), # para evitar overfitting
            
            # FC 2
            nn.Linear(256, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            # Output
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # 1. Tratamento de Dimensão: Garante (Batch, 1, Length)
        # Se vier (Batch, Length), adiciona a dimensão de canal
        if x.ndim == 2:
            x = x.unsqueeze(1)
            
        # 2. Extração de Features
        x = self.features1(x)
        x = self.features2(x)
        
        # 3. Classificação
        x = self.classifier(x)
        
        return x

In [22]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = CNN1D(input_length=input_length, num_classes=num_classes)

exp = DeepLearningExperiment(
    name="cnn1d_7layers_vibration",
    description="1D CNN with 7 layers",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    batch_size=64, 
    lr=1e-3,       
    num_epochs=10
)

results = exp.run()

Input length: 1024, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 0.8494, Val Loss: 0.3777, Time: 92.20s
  [Supervised] Epoch 5/10 Train Loss: 0.1503, Val Loss: 0.0598, Time: 85.62s
  [Supervised] Epoch 10/10 Train Loss: 0.0757, Val Loss: 0.0454, Time: 86.22s
  Result: Acc=0.7629, F1=0.7618

=== Outer Fold 2/4 ===
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 0.8892, Val Loss: 0.5223, Time: 77.86s
  [Supervised] Epoch 5/10 Train Loss: 0.2129, Val Loss: 0.1327, Time: 81.84s
  [Supervised] Epoch 10/10 Train Loss: 0.1147, Val Loss: 0.0513, Time: 81.80s
  Result: Acc=0.9571, F1=0.9570

=== Outer Fold 3/4 ===
[Fold 2] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 0.8853, Val Loss: 0.5622, Time: 82.55s
  [Supervised] Epoch 5/10 Train Loss: 0.2136, Val Loss: 0.1309, Time: 82.42s
  [Supervised] Epoch 10/10 Train Loss: 0.1263, Val Loss: 0.0862, Time: 82.58s
 